# Week 1: Our Approach to Modelling Data
## [Peter Hurley](http://www.sussex.ac.uk/profiles/188689) and [Phil Rooney](http://www.sussex.ac.uk/profiles/252374) 

## Bayesian Probability

### Medical Example

### Saturn Example

### Bayesian Inference

## Probabilistic Programming

Machine learning = black box

#### Neuro example of black box problems

## Generative modelling

## Graphical Models

No single agreed notation but:

Always: 

Nodes represent variables and graph structure represent dependencies

Almost Always: 

Plates indicate replication

Optional Distinctions: 

Continuous vs Discrete
Observed vs Hidden 
Stocastic vs Deterministic
Observed Variables vs Known Property of Experimental Design

### Binary process

### Real world: Sprinkler grass wet

## Pros and Cons


# Examples
-----------------

## Single Variable



In [10]:

import pystan
import numpy as np
import seaborn as sns
from astropy.table import Table
%matplotlib inline


one_vat_model = """

// Inferring a Rate
data { 
  int<lower=1> n; 
  int<lower=0> k;
} 
parameters {
  real<lower=0,upper=1> theta;
} 
model {
  // Prior Distribution for Rate Theta
  theta ~ beta(1, 1);
  
  // Observed Counts
  k ~ binomial(n, theta);
}
"""

k = 10
n = 20

input_data = {'k':10, 'n':20}

samples = pystan.stan(model_code=one_vat_model, data=input_data, iter=10000, chains=2
)


## Schools 

## $p$ values

In [19]:
pearson_model = """
// Pearson Correlation
data { 
  int<lower=0> n;
  vector[2] x[n];
}
parameters {
  vector[2] mu;
  vector<lower=0>[2] lambda;
  real<lower=-1,upper=1> r;
} 
transformed parameters {
  vector<lower=0>[2] sigma;
  cov_matrix[2] T;
  // Reparameterization
  sigma[1] <- inv_sqrt(lambda[1]);
  sigma[2] <- inv_sqrt(lambda[2]);
  T[1,1] <- square(sigma[1]);
  T[1,2] <- r * sigma[1] * sigma[2];
  T[2,1] <- r * sigma[1] * sigma[2];
  T[2,2] <- square(sigma[2]);
}
model {
  // Priors
  mu ~ normal(0, inv_sqrt(.001));
  lambda ~ gamma(.001, .001);
  
  // Data
  x ~ multi_normal(mu, T);
}
"""

pearson_data = np.matrix([[.8,102], [1.0,98], [.5,100], [.9,105], [.7,103], [.4,110], [1.2,99], [1.4,87], [.6,113], [1.1,89], [1.3,93]])
#pearson_data = np.matrix([[.8,102,],[1.0,98,],[.5,100,],[.9,105,],[.7,103,],[.4,110,],[1.2,99,],[1.4,87,],[.6,113,],[1.1,89,],[1.3,93,],[.8,102,],[1.0,98,],[.5,100,],[.9,105,],[.7,103,],[.4,110,],[1.2,99,],[1.4,87,],[.6,113,],[1.1,89,],[1.3,93]])
n = 11
# n = 22
parameters ={"r", "mu", "sigma"}

data = {'x':pearson_data, 'n':n}

samples <- pystan.stan(model_code=pearson_model, data=data, pars=parameters, iter=10000, chains=1)

TypeError: bad operand type for unary -: 'stanfit4anon_model_cbdffd142e6b5f7643dd9a16a8322d7b_7250049168657858168.StanFit4Model'

$\kappa$ Values

In [ ]:
kappa_model ="""
// Kappa Coefficient of Agreement
data { 
  int<lower=0> y[4];
}
parameters {
  // Underlying Rates
  // Rate Objective Method Decides 'one'
  real<lower=0,upper=1> alpha;
  // Rate Surrogate Method Decides 'one' When Objective Method Decides 'one'
  real<lower=0,upper=1> beta;
  // Rate Surrogate Method Decides 'zero' When Objective Method Decides 'zero'
  real<lower=0,upper=1> gamma;
} 
transformed parameters {
  simplex[4] pi;
  real xi;
  real psi;
  real kappa;
  // Probabilities For Each Count
  pi[1] <- alpha * beta;
  pi[2] <- alpha * (1 - beta);
  pi[3] <- (1 - alpha) * (1 - gamma);
  pi[4] <- (1 - alpha) * gamma;
    
  // Derived Measures   
  // Rate Surrogate Method Agrees With the Objective Method
  xi <- alpha * beta + (1 - alpha) * gamma ;
  // Rate of Chance Agreement
  psi <- (pi[1] + pi[2]) * (pi[1] + pi[3]) + (pi[2] + pi[4]) * (pi[3] + pi[4]);  
  // Chance-Corrected Agreement
  kappa <- (xi - psi) / (1 - psi);
}
model {
  alpha ~ beta(1, 1);  // could be removed
  beta ~ beta(1, 1);  // could be removed
  gamma ~ beta(1, 1);  // could be removed
  // Count Data     
  y ~ multinomial(pi);
}
"""
# CHOOSE a data set:
# Influenza 
y = {14, 4, 5, 210}
# Hearing Loss 
# y <- {20, 7, 103, 417}
# Rare Disease
# y <- {0, 0, 13, 157}

data = {'y':y}
parameters = {"kappa", "xi", "psi", "alpha", "beta", "gamma", "pi"}

## Regression

#### Correlated Errors

## Gaussian Mixtures Modelling